# <center>APLU Western Cluster Curricular Analytics Study</center>

In this notebook we analyze the curricula associated with a set of programs at universities in the APLU 
Western Cluster. The cluster of schools is one of the "transformation clusters" formed as a part of APLU's 
[Powered by Publics Initiative]
(https://www.aplu.org/projects-and-initiatives/center-for-public-university-transformation/powered-by-publics) 
<cite data-cite="APLU:18">(APLU, 2019)</cite>. The schools in the Western Cluster include:

- Colorado State University
- Langston University
- Montana State University
- New Mexico State University
- North Dakota State University
- Oklahoma State University
- South Dakota State University
- University of Idaho
- University of Nevada-Reno
- University of Wyoming
- Utah State University

The analyses in this notebook makes use of the [Curricular Analytics toolbox](https://github.com/CurricularAnalytics/CurricularAnalytics.jl) 
built using the Julia programming language  and available as open source software (<cite data-cite="he:19">Heileman, et. al., 2019</cite>). As a starting 
point, you may find it useful to read the [toolbox documenation](https://curricularanalytics.github.io/CurricularAnalytics.jl/latest/), as well as 
the <cite data-cite="he:18">Heileman, et. al., (2018)</cite> paper listed in the [References](#References) 
section below.

If you are interested in joining the community of scholars working to apply curricular analytics, please 
feel free to make suggestions about future development or [contribute](https://curricularanalytics.github.io/CurricularAnalytics.jl/latest/contributing.html) to the 
CurricularAnalytics.jl project.


In order to execute the analyses provided in this notebook, you need to load the following Julia packages.

In [2]:
########## use this to run CA locally
using Pkg
if split(pwd(),Base.Filesystem.path_separator)[end] != "CurricularAnalytics.jl"
    cd("../../CurricularAnalytics.jl/")
end
pkg"activate ."
cd("../CA-Notebooks/Western Cluster-APLU")
############
using CurricularAnalytics
using Glob
using CSV
using DataFrames
using Statistics

┌ Info: Recompiling stale cache file /Users/heileman/.julia/compiled/v1.1/CurricularAnalytics/pJfqI.ji for CurricularAnalytics [593ffa3d-269e-5d81-88bc-c3b6809c35a6]
└ @ Base loading.jl:1184


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-7381912097010965109\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-2439160127008219326\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-7381912097010965109\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

## Create the Data Structures 
First, create an array called `schools` containing the names of the schools in the cluster, given by the subdirectory names in the ./cluster directory.

In [3]:
# read all of the degree plans stored in the ./cluster directory
schools = Array{String,1}()
for path in glob("*", "./cluster")
  push!(schools, replace(path, "./cluster/" => ""))
end

The degree plans for the selected programs at these schools are stored as CSV files in the `./cluster` directory associated with this notebook.  Next, we will read each of these degree plans, and store them as `DegreePlan` objects in a `school_plans` dictionary.

In [4]:
schools_plans = Dict{String, Dict{String, DegreePlan}}()
for school in schools
  plans = Dict{String, DegreePlan}()
  programs = glob("*", "./cluster/$(school)")
  for program in programs
    program_name = replace(split("$(program)", "/")[end], ".csv" => "")
    plans[program_name] = read_csv(program)
    complexity(plans[program_name].curriculum)  # compute the curricular complexity of the degree plan
  end
  schools_plans[school] = plans
end

Curriculum 'Biomedical-Mechanical Engineering' has extraneous requisites:
-MATH 161 has redundant requisite MATH 160
-MECH 105 has redundant requisite MATH 160
-CIVE 260 has redundant requisite MATH 160
-ECE 204 has redundant requisite MATH 161
-MECH 338 has redundant requisite MECH 337

Curriculum 'Mechanical Engineering' has extraneous requisites:
-MECH 105 has redundant requisite MATH 160
-CIVE 260 has redundant requisite MATH 160
-ECE 204 has redundant requisite MATH 161
-MECH 342 has redundant requisite PH 141
-MECH 338 has redundant requisite MECH 337

Curriculum 'Music Education' has extraneous requisites:
-EDUC 350 has redundant requisite EDUC 275
-EDUC 386 has redundant requisite EDUC 275
-EDUC 350 has redundant requisite EDUC 340
-EDUC 386 has redundant requisite EDUC 340
-EDUC 485A has redundant requisite EDUC 450
-EDUC 485B has redundant requisite EDUC 450

Curriculum 'Psychology' has extraneous requisites:
-PY 3213 has redundant requisite PY 1113
-PY 3323 has redundant req

Curriculum 'Curriculum' has extraneous requisites:
-MUSC2030 has redundant requisite MUSC1030
-MUSC2035 has redundant requisite MUSC1035
-MUSC4070 has redundant requisite MUSC2030
-MUSC4070 has redundant requisite MUSC2035

Curriculum 'Curriculum' has extraneous requisites:
-MUSC2030 has redundant requisite MUSC1030
-MUSC2035 has redundant requisite MUSC1035
-MUSC4070 has redundant requisite MUSC2030
-MUSC4070 has redundant requisite MUSC2035

Curriculum 'Curriculum' has extraneous requisites:
-PHYS 2220 has redundant requisite MATH 1210
-ENGR 2030 has redundant requisite MATH 1220
-MAE 3210 has redundant requisite MATH 2210
-MAE 3440 has redundant requisite MAE 2300

Curriculum 'Curriculum' has extraneous requisites:
-MUSC 3180 has redundant requisite MUSC 1110
-MUSC 3180 has redundant requisite MUSC 1120



You can now access and visualize the degree plan associate witha a particular program at a particular school as follows:

In [5]:
dp = schools_plans["colorado-state"]["mechanical-engineering"]
visualize(dp, notebook=true)

┌ Warning: Accessing `scope.id` is deprecated, use `scopeid(scope)` instead.
│   caller = ip:0x0
└ @ Core :-1


WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src=>"http://localhost:8156",:id=>"curriculum",:events=>Dict(:load=>JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"Mechanical Engineering\",\"dp_name\":\"4-Year Plan\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"complexity\":5.0,\"centrality\":0,\"blocking factor\":2,\"delay factor\":3.0},\"nameSub\":\"CHEM 111\",\"id\":1,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"complexity\":5.0,\"centrality\":0,\"blocking factor\":2,\"delay factor\":3.0},\"nameSub\":\"CHEM 112\",\"id\":2,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"complexity\":1.0,\"centrality\":0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"CO 150\",\"id\":3,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"complexity\":33.0,\"centrality\":0,\"blocking factor\":26,\"delay factor\":7.0},\"nameSub\":\"MATH 160\",\"id\":4,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"complexity\":14.0,\"centrality\":0,\"blocking factor\":9,\"delay factor\":5.0},\"nameSub\":\"MECH 103\",\"id\":5,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":4,\"target_id\":6,\"type\":\"prereq\"}],\"name\":\" \",\"metrics\":{\"complexity\":22.0,\"centrality\":87,\"blocking factor\":15,\"delay factor\":7.0},\"nameSub\":\"MATH 161\",\"id\":6,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4,\"target_id\":7,\"type\":\"prereq\"},{\"source_id\":5,\"target_id\":7,\"type\":\"prereq\"},{\"source_id\":8,\"target_id\":7,\"type\":\"prereq\"}],\"name\":\" \",\"metrics\":{\"complexity\":14.0,\"centrality\":64,\"blocking factor\":8,\"delay factor\":6.0},\"nameSub\":\"MECH 105\",\"id\":7,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4,\"target_id\":8,\"type\":\"prereq\"}],\"name\":\" \",\"metrics\":{\"complexity\":28.0,\"centrality\":105,\"blocking factor\":22,\"delay factor\":6.0},\"nameSub\":\"PH 141\",\"id\":8,\"credits\":5,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"complexity\":1.0,\"centrality\":0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"AUCC 3B\",\"id\":9,\"credits\":6,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":4,\"target_id\":10,\"type\":\"prereq\"},{\"source_id\":8,\"target_id\":10,\"type\":\"prereq\"}],\"name\":\" \",\"metrics\":{\"complexity\":12.0,\"centrality\":40,\"blocking factor\":7,\"delay factor\":5.0},\"nameSub\":\"CIVE 260\",\"id\":10,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":6,\"target_id\":11,\"type\":\"prereq\"}],\"name\":\" \",\"metrics\":{\"complexity\":17.0,\"centrality\":64,\"blocking factor\":10,\"delay factor\":7.0},\"nameSub\":\"MATH 261\",\"id\":11,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":7,\"target_id\":12,\"type\":\"prereq\"}],\"name\":\" \",\"metrics\":{\"complexity\":8.0,\"centrality\":16,\"blocking factor\":2,\"delay factor\":6.0},\"nameSub\":\"MECH 200\",\"id\":12,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":7,\"target_id\":13,\"type\":\"prereq\"}],\"name\":\" \",\"metrics\":{\"complexity\":6.0,\"centrality\":16,\"blocking factor\":2,\"delay factor\":4.0},\"nameSub\":\"MECH 201\",\"id\":13,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":8,\"target_id\":14,\"type\":\"prereq\"},{\"source_id\":6,\"target_id\":14,\"type\":\"prereq\"}],\"name\":\" \",\"metrics\":{\"complexity\":10.0,\"centrality\"

## Curricular Complexity

In [15]:
programs = ["animal-science", "mechanical-engineering", "music-education", "psychology"]
for program in programs 
    cc = Array{Real,1}()  # new array for storing curricular complexity values
    for school in schools
      path = "./cluster/$(school)"
      if "$(program).csv" in readdir(path)
        push!(cc, schools_plans[school][program].curriculum.metrics["complexity"][1])
      end
    end
    println("Western Cluster $(program) programs complexity metrics:") 
    println(" Number of programs = $(length(cc))")
    println(" Mean = $(mean(cc))\n STD = $(std(cc))\n Min. = $(minimum(cc))\n Max. = $(maximum(cc))\n\n")
end

Western Cluster animal-science programs complexity metrics:
 Number of programs = 4
 Mean = 178.25
 STD = 88.24728513293388
 Min. = 103.0
 Max. = 306.0


Western Cluster mechanical-engineering programs complexity metrics:
 Number of programs = 10
 Mean = 309.1
 STD = 97.46161637622612
 Min. = 140.0
 Max. = 496.0


Western Cluster music-education programs complexity metrics:
 Number of programs = 9
 Mean = 213.66666666666666
 STD = 52.853571307906904
 Min. = 145.0
 Max. = 295.0


Western Cluster psychology programs complexity metrics:
 Number of programs = 9
 Mean = 76.55555555555556
 STD = 23.324403052978177
 Min. = 40.0
 Max. = 113.0




## References
<a id='References'></a>

APLU, Powered by Publics website (accessed August 15, 2019). https://www.aplu.org/projects-and-initiatives/center-for-public-university-transformation/powered-by-publics

Heileman, G. L., Abdallah, C.T., Slim, A., and Hickman, M. (2018). Curricular analytics: A framework for quantifying the impact of curricular reforms and pedagogical innovations. www.arXiv.org, arXiv:1811.09676 [cs.CY].

Heileman, G. L., Free, H. W., Abar, O. and Thompson-Arjona, W. G, (2019). CurricularAnalytics.jl Toolbox. https://github.com/heileman/CurricularAnalytics.jl.